### Plotly

In [82]:
import plotly.graph_objects as go
import plotly.io as pio

# Define the updated HorizonAnalytics template
HorizonAnalytics = go.layout.Template(
    layout=go.Layout(
        paper_bgcolor='#0d1b2a',  # Background color
        plot_bgcolor='#0d1b2a',  # Background color
        height=800,
        width=800 * 1.618,
        xaxis=dict(
            anchor='y',
            showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.2)',  # Softer grid lines for contrast
            tickfont=dict(
                size=36,  # Consistent with other elements
                family='Montserrat, sans-serif',
                color='#ffffff',
                weight="bold"
            ),
            title=dict(
                text='',
                font=dict(
                    size=48,  # Increase to match other elements
                    family='Montserrat, sans-serif',
                    color='#ffffff',
                    weight="bold"
                )
            ),
            linecolor='#ffffff',  # White axis lines for contrast
            linewidth=2
        ),
        yaxis=dict(
            anchor='x',
            showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.2)',  # Softer grid lines
            tickfont=dict(
                size=36,  # Consistent with x-axis
                family='Montserrat, sans-serif',
                color='#ffffff',
                weight="bold"
            ),
            title=dict(
                text='',
                font=dict(
                    size=48,  # Increase to match x-axis
                    family='Montserrat, sans-serif',
                    color='#ffffff',
                    weight="bold"
                )
            ),
            linecolor='#ffffff',  # White axis lines
            linewidth=2
        ),
        font=dict(
            color='#ffffff',  # White font for all text
            size=36,  # Uniform font size
            family='Montserrat, sans-serif',
            weight="bold"
        ),
        # Refined colorway for better visibility and differentiation
        colorway=["#FFFF00", "#33D7FF", "#A463FF", "#FFD700", 
                  "#ff4081", "#ffc107", "#00c4a0", "#a0aec0"],
        title=dict(
            text='',
            font=dict(
                size=64,  # **Big Boost in Title Size**
                color='#ffffff',
                family='Montserrat, sans-serif',
                weight="bold"
            ),
            x=0.5,  # Center title
            y=0.97  # Push title higher
        )
    ),
    data=dict(
        scatter=[
            go.Scatter(
                line=dict(width=5)  # Increased line width for better visibility
            )
        ]
    )
)

# Register the updated HorizonAnalytics template
pio.templates['HorizonAnalytics'] = HorizonAnalytics
pio.templates.default = 'HorizonAnalytics'

#### https://plotly.com/python/builtin-colorscales/
#### https://www.who.int/data/gho/data/indicators/indicator-details/GHO/prevalence-of-obesity-among-adults-bmi--30-(age-standardized-estimate)-(-)6
#### https://github.com/johan/world.geo.json
#### https://pacific-data.sprep.org/dataset/pacific-island-region-spatial-data
#### https://github.com/topojson/world-atlas/blob/master/README.md

In [83]:
import requests
import pandas as pd

# World Bank API indicator for fertility rate
indicator = "SP.DYN.TFRT.IN"
base_url = f"http://api.worldbank.org/v2"

# Get list of all countries (and groupings) from World Bank
def get_country_metadata():
    country_url = f"{base_url}/country?format=json&per_page=400"
    r = requests.get(country_url)
    total_pages = r.json()[0]['pages']
    country_info = []
    for page in range(1, total_pages + 1):
        r = requests.get(f"{country_url}&page={page}")
        country_info.extend(r.json()[1])
    return {
        c['id']: c['region']['value'] != "Aggregates"
        for c in country_info
    }

# Build a lookup of country_code -> is_country (True/False)
country_lookup = get_country_metadata()

# Get fertility data
data_url = f"{base_url}/country/all/indicator/{indicator}?format=json&per_page=20000"
first_page = requests.get(data_url).json()
total_pages = first_page[0]['pages']

all_data = []
for page in range(1, total_pages + 1):
    page_url = f"{data_url}&page={page}"
    r = requests.get(page_url)
    entries = r.json()[1]
    for entry in entries:
        code = entry['countryiso3code']
        if entry['value'] is not None:
            all_data.append({
                "country_name": entry['country']['value'],
                "country_code": code,
                "year": int(entry['date']),
                "fertility_rate": entry['value'],
                "is_country": country_lookup.get(code, False)
            })

# Save to CSV
df = pd.DataFrame(all_data)
df.to_csv("d_fertility.csv", index=False)
print("Saved d_fertility.csv successfully.")

Saved d_fertility.csv successfully.


## tidy_country_names()

In [84]:
import pandas as pd

def tidy_country_names(csv_path):
    # Define the country name replacements
    replacements = {
        "Democratic People's Republic of Korea": "North Korea",
        "United Republic of Tanzania": "Tanzania",
        "Netherlands (Kingdom of the)": "Netherlands",
        "Venezuela (Bolivarian Republic of)": "Venezuela",
        "Iran (Islamic Republic of)": "Iran",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "Bolivia (Plurinational State of)": "Bolivia",
        "Syrian Arab Republic": "Syria",
        "occupied Palestinian territory, including east Jerusalem": "Palestine",
        "Micronesia (Federated States of)": "Micronesia",
        "Democratic Republic of the Congo": "DR Congo",
        "Republic of Korea": "South Korea",
        "Lao People's Democratic Republic": "Lao",
        "Viet Nam": "Vietnam",
        "Russian Federation": "Russia",
        "United States of America": "United States",
        "Republic of Moldova": "Moldova",
        "Saint Vincent and the Grenadines": "St. Vincent & Grenadines",
        "St. Vincent and the Grenadines": "St. Vincent & Grenadines",
        "West Bank and Gaza": "Palestine",
        "Yemen, Rep.": "Yemen",
        "Venezuela, RB": "Venezuela",
        "Micronesia, Fed. Sts.": "Micronesia",
        "Macao SAR, China": "Macao",
        "Lao PDR": "Lao",
        "Kyrgyz Republic": "Kyrgyzstan",
        "Korea, Rep.": "South Korea",
        "Korea, Dem. People's Rep.": "North Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong",
        "Gambia, The": "Gambia",
        "Egypt, Arab Rep.": "Egypt",
        "Congo, Rep.": "Congo",
        "Congo, Dem. Rep.": "DR Congo",
        "Bahamas, The": "The Bahamas",
        "St. Martin (French part)": "St. Martin (French)",
        "St. Martin (Dutch part)": "St. Martin (Dutch)"

    }

    # Read CSV
    df = pd.read_csv(csv_path)

    # Replace country names
    df["country_name"] = df["country_name"].replace(replacements)

    # Overwrite CSV
    df.to_csv(csv_path, index=False)

    print(f"✅ Tidied and saved: {csv_path}")

tidy_country_names("d_fertility.csv")

✅ Tidied and saved: d_fertility.csv


In [85]:
import pandas as pd

# Load the full dataset
df = pd.read_csv("d_fertility.csv")

# Filter for actual countries
df_countries = df[df["is_country"] == True]

# Save to new CSV
df_countries.to_csv("d_country_fertility.csv", index=False)
print("Saved d_country_fertility.csv successfully.")

Saved d_country_fertility.csv successfully.


## filter_csv()

In [86]:
import pandas as pd
from typing import Union, List

def filter_csv(input_path="d_fertility.csv", output_path="d_country_fertility.csv", years: Union[int, List[int], None] = None):
    # Load dataset
    df = pd.read_csv(input_path)

    # Filter for actual countries
    df_filtered = df[df["is_country"] == True]

    # Optional: filter by year(s)
    if years is not None:
        if isinstance(years, int):
            years = [years]
        df_filtered = df_filtered[df_filtered["year"].isin(years)]

    # Save filtered dataset
    df_filtered.to_csv(output_path, index=False)
    print(f"Saved {output_path} successfully.")

## order_csv()

In [87]:
import pandas as pd

def order_csv(csv_path, column_name, direction="asc"):
    # Read the CSV
    df = pd.read_csv(csv_path)

    # Check sort direction
    ascending = True if direction.lower() == "asc" else False

    # Sort the DataFrame
    df_sorted = df.sort_values(by=column_name, ascending=ascending)

    # Save back to the same file
    df_sorted.to_csv(csv_path, index=False)

    print(f"✅ Sorted by '{column_name}' ({direction}) and saved: {csv_path}")

## load_world_geojson()

In [88]:
import geopandas as gpd
import requests

def load_world_geojson(source='local'):
    """
    Load world map from a local simplified GeoJSON or a simplified online version.

    Parameters:
        source (str): 'local' or 'web'

    Returns:
        GeoDataFrame or dict: GeoDataFrame if local, dict if web
    """
    if source == 'local':
        local_geojson_path = "../ne_10m_admin_0_countries_lakes/world_simplified.geojson"
        try:
            print(f"📁 Loading local simplified GeoJSON from: {local_geojson_path}")
            return gpd.read_file(local_geojson_path)
        except FileNotFoundError:
            raise FileNotFoundError(f"❌ GeoJSON file not found at: {local_geojson_path}")

    elif source == 'web':
        print("🌍 Loading simplified GeoJSON from Johan's GitHub...")
        geojson_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
        response = requests.get(geojson_url)
        response.raise_for_status()
        return response.json()

    else:
        raise ValueError("Invalid source. Use 'local' or 'web'.")

In [89]:
filter_csv(years=2022)
order_csv("d_country_fertility.csv", column_name="fertility_rate", direction="asc")
world_geojson = load_world_geojson('web')

Saved d_country_fertility.csv successfully.
✅ Sorted by 'fertility_rate' (asc) and saved: d_country_fertility.csv
🌍 Loading simplified GeoJSON from Johan's GitHub...


In [90]:
small_countries_coords = {
    'TON': (-21.2, -175.2),
    'WSM': (-13.8, -172.1),
    'NRU': (-0.52, 166.93),
    'KIR': (1.87, 173.0),
    'PLW': (7.5, 134.5),
    'MHL': (7.1, 171.0),
    'FSM': (6.9, 158.2),
    'VUT': (-16.0, 167.0),
    'TUV': (-7.1, 179.2),
    'NIU': (-19.05, -169.86),
    'COK': (-21.2, -159.8),
    'TKL': (-9.2, -171.8),
    'ASM': (-14.3, -170.7),
    'CPV': (15.1, -23.6),
    'STP': (0.2, 6.6),
    'COM': (-11.6, 43.3),
    'LUX': (49.8, 6.1),
    'AND': (42.5, 1.5),
    'LCA': (13.9, -60.98),
    'ATG': (17.1, -61.8),
    'DMA': (15.4, -61.4),
    'VCT': (13.25, -61.2),
    'KNA': (17.3, -62.7),
    'BRB': (13.1, -59.6),
    'BMU': (32.3, -64.8),
    'MDV': (3.2, 73.2),
    'SGP': (1.35, 103.8),
    'BHR': (26.0, 50.5),
    'QAT': (25.3, 51.5),
    'DJI': (11.8, 42.6),
    'GMB': (13.5, -15.5),
    'SYC': (-4.6, 55.5),
    'MUS': (-20.2, 57.5),
    'GNQ': (1.6, 10.5),
    'GNB': (11.8, -15.2),
    'SLB': (-9.6, 160.2),
    'PYF': (-17.6797, -149.4068),  # French Polynesia (Tahiti approx.)
    'HKG': (22.3, 114.2),
    'PRI': (18.2, -66.5),
    'VGB': (18.4, -64.6),
    'MAC': (22.2, 113.6),           # Macao
    'MLT': (35.9, 14.5),             # Malta 
    'ABW': (12.5, -69.97),     # Aruba 
    'CUW': (12.2, -69.0),      # Curaçao
    'CYP': (35.1, 33.4),       # Cyprus
    'JAM': (18.1, -77.3),      # Jamaica
    'BHS': (25.03, -77.4),     # The Bahamas
    'CHI': (49.5, -2.5),       # Channel Islands (approx. between Jersey and Guernsey)
    'GIB': (36.1, -5.35),       # Gibraltar
    'VIR': (18.34, -64.89),    # US Virgin Islands
    'NCL': (-22.27, 166.44),   # New Caledonia
    'FRO': (62.0, -6.79),      # Faroe Islands
    'MAF': (18.07, -63.05),    # St Martin (French)
    'SXM': (18.04, -63.07),    # St Martin (Dutch)
    'GUM': (13.44, 144.79),    # Guam
    'OMN': (21.5, 55.9)        # Oman (added as it's actually a full country)
}

## map_frames ()

In [111]:
import pandas as pd
import plotly.graph_objects as go
import os
import shutil

def map_frames(
    csv_filename,
    world_geojson,
    small_countries_coords,
    metric='obesity_rate', 
    plot_height=1080,
    plot_width=1920,
    color_min='255,255,0',
    color_mid='255,165,0',
    color_max='255,0,0',
    parent_folder='2023-03_obesity',
    output_folder='frames_world',
    clear=True,
    marker_size=10,
    min_color_percentile=5,
    max_color_percentile=95
):
    # Load data
    df = pd.read_csv(csv_filename)
    latest_year = df['year'].max()
    df = df[df['year'] == latest_year].reset_index(drop=True)

    # Compute color scale values based on input percentiles
    color_min_value = df[metric].quantile(min_color_percentile / 100)
    color_max_value = df[metric].quantile(max_color_percentile / 100)
    color_mid_value = df[metric].mean()

    print(f"🎨 Color scale range for '{metric}':")
    print(f"   Min (p{min_color_percentile}): {color_min_value:.2f}")
    print(f"   Mid (mean): {color_mid_value:.2f}")
    print(f"   Max (p{max_color_percentile}): {color_max_value:.2f}")

    geojson = world_geojson

    all_countries = pd.DataFrame({
        'country_code': [
            feature.get('id') or feature['properties'].get('iso_a3')
            for feature in geojson['features']
        ]
    })

    # Convert small country coords to DataFrame
    small_countries = pd.DataFrame([
        {'country_code': code, 'lat': lat, 'lon': lon}
        for code, (lat, lon) in small_countries_coords.items()
    ])

    # Set output path
    output_path = f"/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/{parent_folder}/{output_folder}"
    if clear and os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    for i in range(1, len(df) + 1):
        df_partial = df.iloc[:i]
        df_merged = all_countries.merge(
            df_partial[['country_code', 'country_name', metric]],
            on='country_code', how='left'
        )

        small_data = small_countries.merge(
            df_partial[['country_code', metric]],
            on='country_code',
            how='left'
        )
        small_data = small_data[small_data[metric].notna()]

        fig = go.Figure()

        # Choropleth base layer
        fig.add_trace(go.Choropleth(
            geojson=geojson,
            locations=df_merged['country_code'],
            z=df_merged[metric],
            locationmode='ISO-3',
            colorscale=[
                [0.0, f'rgb({color_min})'],
                [0.5, f'rgb({color_mid})'],
                [1.0, f'rgb({color_max})']
            ],
            zmin=color_min_value,
            zmax=color_max_value,
            colorbar_title=metric.replace("_", " ").capitalize(),
            text=df_merged['country_name'],
            hoverinfo='location+z+text',
            marker_line_color='white',
            marker_line_width=1.5,
        ))

        # Small country markers
        fig.add_trace(go.Scattergeo(
            lon=small_data['lon'],
            lat=small_data['lat'],
            mode='markers',
            marker=dict(
                size=marker_size,
                color=small_data[metric],
                colorscale=[
                    [0.0, f'rgb({color_min})'],
                    [0.5, f'rgb({color_mid})'],
                    [1.0, f'rgb({color_max})']
                ],
                cmin=color_min_value,
                cmax=color_max_value,
                colorbar=None,
                line=dict(width=3.5, color='white'),
                opacity=1.0
            ),
            hoverinfo='skip',
            showlegend=False
        ))

        # Global map settings
        fig.update_geos(
            showcountries=True,
            countrycolor="white",
            showframe=False,
            showcoastlines=False,
            showlakes=False,
            bgcolor='rgba(0,0,0,0)',
            lataxis_range=[-60, 90],
            projection_rotation={"lon": 40}
        )

        fig.update_layout(
            margin={"r": 0, "t": 0, "l": 0, "b": 0},
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=plot_height,
            width=plot_width,
            coloraxis_colorbar=dict(
                len=0.5,
                yanchor='middle',
                y=0.5
            )
        )

        # Save frame
        filepath = f"{output_path}/{i:04d}.png"
        fig.write_image(filepath, scale=1)

        current_country = df_partial.iloc[-1]['country_name']
        print(f"✅ Saved {filepath} – added: {current_country}")

In [112]:
map_frames(
    csv_filename="d_country_fertility.csv",
    world_geojson=world_geojson,
    small_countries_coords=small_countries_coords,
    metric="fertility_rate",
    plot_height=1080,
    plot_width=1920,
    color_min = "143,217,251",  # light blue
    color_mid = "0,0,255",  # medium blue
    color_max = "17,17,132",     # dark blue
    parent_folder="2023-03_fertility",
    output_folder="frames_world",
    clear=True,
    marker_size=18,
    min_color_percentile=5,
    max_color_percentile=95
)

🎨 Color scale range for 'fertility_rate':
   Min (p5): 1.25
   Mid (mean): 2.48
   Max (p95): 4.66
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0001.png – added: Hong Kong
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0002.png – added: South Korea
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0003.png – added: Puerto Rico
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0004.png – added: British Virgin Islands
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0005.png – added: Singapore
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0006.png – added: Macao
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world/0007.png – added: Malta
✅ Saved /U

## text_frames ()

In [125]:
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os
import shutil

def interpolate_color(val, min_val, mid_val, max_val, rgb_min, rgb_mid, rgb_max):
    if val <= mid_val:
        ratio = (val - min_val) / (mid_val - min_val) if mid_val > min_val else 0
    else:
        ratio = 0.5 + 0.5 * ((val - mid_val) / (max_val - mid_val)) if max_val > mid_val else 1

    ratio = max(0, min(1, ratio))  # Ensure ratio is within [0, 1]

    if ratio <= 0.5:
        r_ratio = ratio * 2
        return tuple([
            int(rgb_min[i] + r_ratio * (rgb_mid[i] - rgb_min[i]))
            for i in range(3)
        ])
    else:
        r_ratio = (ratio - 0.5) * 2
        return tuple([
            int(rgb_mid[i] + r_ratio * (rgb_max[i] - rgb_mid[i]))
            for i in range(3)
        ])

def save_text_component(text, text_color, outline_color, font, image_width, image_height, x, y, filepath):
    # Create transparent image
    img = Image.new('RGBA', (image_width, image_height), (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)

    # Draw outline
    offset = 2
    for dx in [-offset, 0, offset]:
        for dy in [-offset, 0, offset]:
            if dx == 0 and dy == 0:
                continue
            draw.text((x + dx, y + dy), text, font=font, fill=outline_color)
    
    # Draw main text
    draw.text((x, y), text, font=font, fill=text_color)
    img.save(filepath)
    print(f"🖼️ Saved {filepath}")

def extract_text(
    csv_filename,
    parent_folder='2023-03_obesity',
    output_folder='frames_text',
    output_folder_suffix='',
    metric='fertility_rate',
    font_type='ExtraBold',
    font_size=100,
    image_width=1920,
    image_height=1080,
    color_min='143,217,251',
    color_mid='0,0,255',
    color_max='17,17,132',
    min_color_percentile=5,
    max_color_percentile=95,
    rank_border=112,
    clear=True
):
    # Parse colors
    rgb_min = tuple(map(int, color_min.split(',')))
    rgb_mid = tuple(map(int, color_mid.split(',')))
    rgb_max = tuple(map(int, color_max.split(',')))

    # Output path
    full_output_folder = os.path.join(parent_folder, output_folder + output_folder_suffix)
    base_output_path = os.path.join(
        "/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics",
        full_output_folder
    )
    
    if clear and os.path.exists(base_output_path):
        shutil.rmtree(base_output_path)
    os.makedirs(base_output_path, exist_ok=True)

    # Create subfolders for each text type
    text_types = ['title', 'value', 'rank', 'country']
    text_paths = {text_type: os.path.join(base_output_path, text_type) for text_type in text_types}
    for path in text_paths.values():
        os.makedirs(path, exist_ok=True)

    # Load and filter data
    df = pd.read_csv(csv_filename)
    latest_year = df['year'].max()
    df = df[df['year'] == latest_year].reset_index(drop=True)

    # Compute thresholds
    color_min_value = df[metric].quantile(min_color_percentile / 100)
    color_max_value = df[metric].quantile(max_color_percentile / 100)
    color_mid_value = df[metric].mean()

    # Add ranks
    df['rank'] = df[metric].rank(method='min', ascending=False).astype(int)

    # Load font
    font_path = os.path.join("..", "Montserrat", f"Montserrat-{font_type}.ttf")
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Montserrat font '{font_type}' not found. Using default font.")
        font = ImageFont.load_default()

    for i in range(1, len(df) + 1):
        row = df.iloc[i - 1]
        country = row['country_name']
        rate = round(row[metric], 2)
        rank = row['rank']

        # Determine text color
        text_color = interpolate_color(
            rate, color_min_value, color_mid_value, color_max_value,
            rgb_min, rgb_mid, rgb_max
        )

        # Determine outline color based on rank
        if rank <= rank_border:
            outline_color = (255, 255, 255)
        else:
            outline_color = (255, 255, 255)

        # Define text components
        texts = {
            'title': "KIDS / WOMAN",
            'value': str(rate),
            'rank': f"#{rank}",
            'country': country.upper()
        }

        # Save each text component individually, all left-indented
        x = 100  # Left-indented
        y_positions = {
            'title': 100,
            'value': 250,
            'rank': 400,
            'country': 550
        }

        for text_type, text in texts.items():
            y = y_positions[text_type]
            filepath = os.path.join(text_paths[text_type], f"{i:04d}.png")
            save_text_component(text, text_color, outline_color, font, image_width, image_height, x, y, filepath)

In [126]:
extract_text(
    csv_filename="d_country_fertility.csv",
    parent_folder="2023-03_fertility",
    output_folder="frames_text",
    output_folder_suffix="",
    metric="fertility_rate",
    font_type="ExtraBold",
    font_size=100,
    image_width=1920,
    image_height=1080,
    color_min = "143,217,251",  # light blue
    color_mid = "0,0,255",  # medium blue
    color_max = "17,17,132",     # dark blue
    min_color_percentile=5,
    max_color_percentile=95,
    rank_border=112,
    clear=True
)

🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/title/0001.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/value/0001.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/rank/0001.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/country/0001.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/title/0002.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/value/0002.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/rank/0002.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_text/country/0002.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_t

In [115]:
import pandas as pd
import os
import shutil
import requests
from PIL import Image
from io import BytesIO
from pycountry import countries

def save_flag_frames(
    csv_filename,
    parent_folder='2023-03_obesity',
    output_folder='frames_flag',
    image_width=1920,
    image_height=1080,
    clear=True
):
    # Output path
    output_path = os.path.join(
        "/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics",
        parent_folder,
        output_folder
    )

    if clear and os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    # Load data and preserve original order
    df = pd.read_csv(csv_filename)
    latest_year = df['year'].max()
    df = df[df['year'] == latest_year].reset_index(drop=True)
    df = df[df["is_country"] == True]

    # Create ISO-3 to ISO-2 mapping
    iso_map = {c.alpha_3: c.alpha_2.lower() for c in countries if hasattr(c, 'alpha_3')}

    for i in range(len(df)):
        row = df.iloc[i]
        code3 = row['country_code']
        country = row['country_name']
        code2 = iso_map.get(code3)

        if not code2:
            print(f"⚠️ No ISO-2 code for {country} ({code3})")
            continue

        url = f"https://flagcdn.com/w320/{code2}.png"
        try:
            response = requests.get(url)
            response.raise_for_status()
            flag = Image.open(BytesIO(response.content)).convert("RGBA")
        except Exception as e:
            print(f"❌ Failed to load flag for {country} ({code3}): {e}")
            continue

        # Resize and center the flag on a transparent canvas
        flag_aspect = flag.width / flag.height
        target_height = image_height
        target_width = int(target_height * flag_aspect)
        flag = flag.resize((target_width, target_height), Image.LANCZOS)

        img = Image.new("RGBA", (image_width, image_height), (0, 0, 0, 0))
        x = (image_width - flag.width) // 2
        y = (image_height - flag.height) // 2
        img.paste(flag, (x, y), flag)

        filepath = os.path.join(output_path, f"{i+1:04d}.png")
        img.save(filepath)
        print(f"🏁 Saved {filepath} – {country}")

# Example usage
save_flag_frames(
    csv_filename='d_country_fertility.csv',
    parent_folder='2023-03_fertility',
    output_folder='frames_flag' 
)

🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0001.png – Hong Kong
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0002.png – South Korea
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0003.png – Puerto Rico
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0004.png – British Virgin Islands
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0005.png – Singapore
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0006.png – Macao
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0007.png – Malta
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_flag/0008.png – Spain
🏁 Saved /Users/arya/Documents/Adobe/Premiere Pro/Ho

## history_frames()

In [131]:
def map_frames_hardcoded(
    csv_filename,
    world_geojson,
    small_countries_coords,
    year,
    index,
    metric='obesity_rate',
    plot_height=1080,
    plot_width=1920,
    parent_folder='2023-03_obesity',
    output_folder='frames_world',
    clear=False,
    marker_size=10
):
    import pandas as pd
    import plotly.graph_objects as go
    import os
    import shutil

    # Hardcoded color scale values
    color_min = "143,217,251"  # light blue
    color_mid = "0,0,255"  # medium blue
    color_max = "17,17,132"     # dark blue
    color_min_value = 1.25
    color_mid_value = 2.47
    color_max_value = 4.66

    # Load data
    df = pd.read_csv(csv_filename)
    df = df[df['year'] == year].reset_index(drop=True)

    geojson = world_geojson

    all_countries = pd.DataFrame({
        'country_code': [
            feature.get('id') or feature['properties'].get('iso_a3')
            for feature in geojson['features']
        ]
    })

    # Convert small country coords dict to DataFrame
    small_countries = pd.DataFrame([
        {'country_code': code, 'lat': lat, 'lon': lon}
        for code, (lat, lon) in small_countries_coords.items()
    ])

    # Prepare output directory
    output_path = f"/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/{parent_folder}/{output_folder}"
    if clear and os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    # Merge full data
    df_merged = all_countries.merge(
        df[["country_code", "country_name", metric]],
        on="country_code", how="left"
    )

    small_data = small_countries.merge(
        df[["country_code", metric]],
        on="country_code",
        how="left"
    )
    small_data = small_data[small_data[metric].notna()]

    fig = go.Figure()

    # Choropleth
    fig.add_trace(go.Choropleth(
        geojson=geojson,
        locations=df_merged['country_code'],
        z=df_merged[metric],
        locationmode='ISO-3',
        colorscale=[
            [0.0, f'rgb({color_min})'],
            [0.5, f'rgb({color_mid})'],
            [1.0, f'rgb({color_max})']
        ],
        zmin=color_min_value,
        zmax=color_max_value,
        colorbar_title=metric.replace("_", " ").capitalize(),
        text=df_merged['country_name'],
        hoverinfo='location+z+text',
        marker_line_color='white',
        marker_line_width=1.5,
    ))

    # Small country markers
    fig.add_trace(go.Scattergeo(
        lon=small_data['lon'],
        lat=small_data['lat'],
        mode='markers',
        marker=dict(
            size=marker_size,
            color=small_data[metric],
            colorscale=[
                [0.0, f'rgb({color_min})'],
                [0.5, f'rgb({color_mid})'],
                [1.0, f'rgb({color_max})']
            ],
            cmin=color_min_value,
            cmax=color_max_value,
            colorbar=None,
            line=dict(width=3.5, color='white'),
            opacity=1.0
        ),
        hoverinfo='skip',
        showlegend=False
    ))

    # Global map layout
    fig.update_geos(
        showcountries=True,
        countrycolor="white",
        showframe=False,
        showcoastlines=False,
        showlakes=False,
        bgcolor='rgba(0,0,0,0)',
        lataxis_range=[-60, 90],
        projection_rotation={"lon": 40}
    )

    fig.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        height=plot_height,
        width=plot_width,
        coloraxis_colorbar=dict(
            len=0.5,
            yanchor='middle',
            y=0.5
        )
    )

    # Save frame
    filepath = f"{output_path}/{index:04d}.png"
    fig.write_image(filepath, scale=1)
    print(f"✅ Saved {filepath} for year {year}")

In [132]:
import pandas as pd
import json

# Set file path and read dataset
csv_filename = "d_fertility.csv"
df = pd.read_csv(csv_filename)
df = df[df["is_country"] == True]

# Get all unique years in order
years = sorted(df["year"].dropna().unique())

# Generate one map per year
for i, year in enumerate(years, start=1):
    map_frames_hardcoded(
        csv_filename="d_fertility.csv",
        world_geojson=world_geojson,
        small_countries_coords=small_countries_coords,
        year=year,
        index=i,
        metric="fertility_rate",
        plot_height=1080,
        plot_width=1920,
        parent_folder="2023-03_fertility",
        output_folder="frames_world_history",
        clear=False,  # important to avoid deleting output folder each time
        marker_size=18
    )


✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0001.png for year 1960
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0002.png for year 1961
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0003.png for year 1962
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0004.png for year 1963
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0005.png for year 1964
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0006.png for year 1965
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world_history/0007.png for year 1966
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_world

In [135]:
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os
import shutil

def extract_text_all_years(
    csv_filename,
    parent_folder='2023-03_obesity',
    output_folder='frames_years_only',
    font_type='ExtraBold',
    font_size=200,
    image_width=1920,
    image_height=1080,
    alignment='center',
    clear=True
):
    # Colors
    text_color = (0, 0, 0, 0)  # black
    outline_color = (255, 255, 255)  # white

    # Output path
    output_path = os.path.join(
        "/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics",
        parent_folder,
        output_folder
    )
    if clear and os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    # Load data
    df = pd.read_csv(csv_filename)

    # Load font
    font_path = os.path.join("..", "Montserrat", f"Montserrat-{font_type}.ttf")
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Montserrat font '{font_type}' not found. Using default font.")
        font = ImageFont.load_default()

    # Unique years (reverse order)
    years = sorted(df['year'].unique(), reverse=True)

    for i, year in enumerate(years, start=1):
        text = str(year)

        # Create transparent image
        img = Image.new('RGBA', (image_width, image_height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(img)

        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        if alignment == 'left':
            x = 100
        elif alignment == 'right':
            x = image_width - text_width - 100
        else:
            x = (image_width - text_width) / 2

        y = (image_height - text_height) / 2

        # Outline
        offset = 4
        for dx in [-offset, 0, offset]:
            for dy in [-offset, 0, offset]:
                if dx == 0 and dy == 0:
                    continue
                draw.text((x + dx, y + dy), text, font=font, fill=outline_color)

        # Main text
        draw.text((x, y), text, font=font, fill=text_color)

        filepath = os.path.join(output_path, f"{i:04d}.png")
        img.save(filepath)
        print(f"🖼️ Saved {filepath}")


In [136]:
extract_text_all_years(
    csv_filename='d_fertility.csv',
    parent_folder='2023-03_fertility',
    output_folder='frames_years_only',
    font_type='ExtraBold',
    font_size=200,
    image_width=1920,
    image_height=1080,
    alignment='center',
    clear=True
)

🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0001.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0002.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0003.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0004.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0005.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0006.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0007.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_years_only/0008.png
🖼️ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_fertility/frames_yea